#  Keras Intro: Fully Connected Network

Keras Documentation: https://keras.io

In this notebook we explore how to use Keras to implement Deep Fully Connected models

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

## Shallow and Deep Networks

In [ ]:
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=1000, noise=0.1, random_state=0)

plt.plot(X[y==0, 0], X[y==0, 1], 'ob', alpha=0.5)
plt.plot(X[y==1, 0], X[y==1, 1], 'xr', alpha=0.5)
plt.legend(['0', '1'])

In [ ]:
X.shape

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3,
                                                    random_state=42)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD, Adam

### Shallow Model

In [ ]:
model = Sequential()
model.add(Dense(1, input_shape=(2,), activation='sigmoid'))
model.compile(Adam(lr=0.05), 'binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=200, verbose=0)

In [ ]:
results = model.evaluate(X_test, y_test)

In [ ]:
results

In [ ]:
print("The Accuracy score on the Train set is:\t{:0.3f}".format(results[1]))

In [ ]:
def plot_decision_boundary(model, X, y):
    amin, bmin = X.min(axis=0) - 0.1
    amax, bmax = X.max(axis=0) + 0.1
    hticks = np.linspace(amin, amax, 101)
    vticks = np.linspace(bmin, bmax, 101)
    
    aa, bb = np.meshgrid(hticks, vticks)
    ab = np.c_[aa.ravel(), bb.ravel()]
    
    c = model.predict(ab)
    cc = c.reshape(aa.shape)

    plt.figure(figsize=(12, 8))
    plt.contourf(aa, bb, cc, cmap='bwr', alpha=0.2)
    plt.plot(X[y==0, 0], X[y==0, 1], 'ob', alpha=0.5)
    plt.plot(X[y==1, 0], X[y==1, 1], 'xr', alpha=0.5)
    plt.legend(['0', '1'])

In [ ]:
plot_decision_boundary(model, X, y)

### Deep model

In [ ]:
model = Sequential()
model.add(Dense(4, input_shape=(2,), activation='tanh'))
model.add(Dense(2, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))
model.compile(Adam(lr=0.05), 'binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=100, verbose=0)

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
y_train_pred = model.predict_classes(X_train)
y_test_pred = model.predict_classes(X_test)

print("The Accuracy score on the Train set is:\t{:0.3f}".format(accuracy_score(y_train, y_train_pred)))
print("The Accuracy score on the Test set is:\t{:0.3f}".format(accuracy_score(y_test, y_test_pred)))

In [ ]:
plot_decision_boundary(model, X, y)

## Multiclass classification

### The Iris dataset

In [ ]:
df = pd.read_csv('../data/iris.csv')

In [ ]:
import seaborn as sns
sns.pairplot(df, hue="target")

In [ ]:
df.head()

In [ ]:
X = df.drop('target', axis=1)
X.head()

In [ ]:
target_names = df['target'].unique()
target_names

In [ ]:
target_dict = {n:i for i, n in enumerate(target_names)}
target_dict

In [ ]:
y= df['target'].map(target_dict)
y.head()

In [ ]:
from keras.utils.np_utils import to_categorical

In [ ]:
y_cat = to_categorical(y)

In [ ]:
y_cat[:10]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X.values, y_cat,
                                                    test_size=0.2,
                                                    random_state=0)

In [ ]:
model = Sequential()
model.add(Dense(3, input_shape=(4,), activation='softmax'))
model.compile(Adam(lr=0.1),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=20, validation_split=0.1)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred[:5]

In [ ]:
y_test_class = np.argmax(y_test, axis=1)
y_pred_class = np.argmax(y_pred, axis=1)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test_class, y_pred_class))

In [ ]:
def pretty_confusion_matrix(y_true, y_pred, labels=target_names):
    cm = confusion_matrix(y_true, y_pred)
    pred_labels = ['Predicted '+ l for l in labels]
    df = pd.DataFrame(cm, index=labels, columns=pred_labels)
    return df

In [ ]:
pretty_confusion_matrix(y_test_class, y_pred_class)

### Exercise 1

- load the churn dataset (../data/churn.csv)
- assign the Churn column to a variable called `y`
- assign the other columns to a variable called `features`
- separate numerical columns with `features.select_dtypes`
- split data into train/test with test_size=0.3 and random_state=42
- classify the resulting data using a Fully Connected neural network
- try to improve the score changing:
    - number of layers
    - number of nodes per layer
    - optimizer
    - learning rate
- what's the best score you can reach on the test set?
- is the score better than always predicting no-churn?
- is the model overfitting?

In [ ]:
df = pd.read_csv('../data/churn.csv', )

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
y = (df['Churn'] == 'Yes').values
features = df.drop('Churn', axis=1)

In [ ]:
X = features.select_dtypes(include=['number'])

In [ ]:
X.info()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
model = Sequential()
model.add(Dense(10, input_shape=X_train.shape[1:], activation='relu'))
model.add(Dense(5, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(Adam(lr=0.1),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
h = model.fit(X_train, y_train, epochs=10, validation_split=0.1)

In [ ]:
pd.DataFrame(h.history).plot()

In [ ]:
model.evaluate(X_train, y_train)

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
pd.Series(y_test).value_counts() /len(y_test)